# Slicing modelisation of 5G networks

## Importing libraries

In [1]:
import numpy as np

## Part 1

### Exercise 1

Let $P(k)$ be the probability of having $k$ clients in a queue $M / M / S / S$.
As seen in class, this probability is equal to:
$$
P(k) = \frac{\frac{\rho^k}{k!}}{\sum_{j = 0}^S \frac{\rho^j}{j!}}.
$$

Thus, it is clear that $P(k)$ is defined by the following recurrence relation:

\begin{align}
P(k) &= \frac{\rho}{k} P(k - 1) \\
&= \frac{\rho^2}{k (k - 1)} P(k - 2) \\
&\vdots \\
&= \frac{\rho^k}{k!} P(0)
\end{align}

So, we have one degree of freedom defined by the value of $P(0)$.
Since $\sum_{k = 0}^S P(k) = 1$, the value of $P(0)$ is simply a normalizing constant, which makes sense when looking at the whole equation above.
Therefore, we can compute every $P(k)$ without factorials by simply using the recurrence relation and, after that, normalize all values such that their sum is equal to 1.
Moreover, to find the mean number of clients in the system, we can use the expectation formula, i.e.,
$$
\mu_N = \mathbb{E}[P(k)] = \sum_{k = 0}^S k P(k)
$$

In [2]:
def client_probs(rho, S):
    Pk = [1]
    
    for k in range(1, S + 1):
        prob = (rho / k) * Pk[-1]
        Pk.append(prob)

    Pk = np.array(Pk)
    return Pk / Pk.sum()

In [3]:
def mean_number_waiting_customers(arrival_rate, service_rate, nb_of_servers):
    rho = arrival_rate / service_rate
    S = nb_of_servers

    Pk = client_probs(rho, S)
    return np.dot(np.arange(S + 1), Pk)

### Exercise 2

In [4]:
def erlang_b(rho, S):
    if S == 0:
        return 1
        
    erlang_inv = 1 + S / (rho * erlang_b(rho, S - 1))
    return 1 / erlang_inv

In [14]:
S = 4
lamb = 6
mu = 12

rho = lamb / mu

# Sanity check
print(np.abs(erlang_b(rho, S) - client_probs(rho, S)[S]) < 1e-10)

True


In [19]:
print(f"{rho = }, {S = }")
print(f"Erl_B = {erlang_b(rho, S):.4f}")
print(f"mean clients = {mean_number_waiting_customers(lamb, mu, S):.4f}")

rho = 0.5, S = 4
Erl_B = 0.0016
mean clients = 0.4992


We notice that the mean number of clients is approximately equal to $\rho$. This makes sense because in a $M / M / \infty$, we can find the mean number as exactly $\rho$ as shown below:

\begin{align}
\lim_{S \to \infty} \mathbb{E}[P(k)] &= \lim_{S \to \infty} \sum_{k = 0}^S k P(k) \\
&= \lim_{S \to \infty} \sum_{k = 0}^S k \frac{\rho^k}{k!} P(0) \\
&= \lim_{S \to \infty} \rho P(0) \sum_{k = 1}^S \frac{\rho^{k - 1}}{(k - 1)!} \\
&= \rho \lim_{S \to \infty} P(0) \sum_{k = 0}^{S - 1} \frac{\rho^{k}}{k!} \\
&= \rho \lim_{S \to \infty} \frac{1}{\sum_{j = 0}^{S} \frac{\rho^{j}}{j!}} \sum_{k = 0}^{S - 1} \frac{\rho^{k}}{k!} \\
&= \rho.
\end{align}

Since the Erlang-B is very small ($10^{-3}$), our queue behaves like a $M / M / \infty$ because there is almost no blockage.